In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

In [ ]:
import os
import wandb
#os.environ['WANDB_MODE'] = 'dryrun'
os.environ['WANDB_NOTEBOOK_NAME'] = 'data.ipynb'
wandb.init("sky-eye-full")
conf = wandb.config

In [ ]:
from pprint import pprint
pprint(dict(conf))

In [ ]:
from glob import glob
train_dir = '/home/jovyan/work/datasets/xview/train'
test_dir = '/home/jovyan/work/datasets/xview/test'

In [ ]:
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
from torch import nn
from xv.util import vis_im_mask
import segmentation_models_pytorch as smp

In [ ]:
import albumentations as al

augment = al.Compose([
        al.HorizontalFlip(p=conf.aug_prob),
        al.VerticalFlip(p=conf.aug_prob),
        al.RandomRotate90(p=conf.aug_prob),
        al.Transpose(p=conf.aug_prob),
        al.GridDistortion(p=conf.aug_prob, distort_limit=.2),
        al.ShiftScaleRotate(p=conf.aug_prob),
        al.RandomBrightnessContrast(p=conf.aug_prob)
])

In [ ]:
from xv.nn.solaris.model_io import get_model
from xv.nn.nets import DownscaleLayer, XVNet
from torchvision.models.resnet import BasicBlock, Bottleneck
import segmentation_models_pytorch as smp


block_types = {
    'bottleneck': Bottleneck,
    'basic': BasicBlock
}

segmentation_types = {
    'PSPNet': smp.PSPNet,
    'FPN': smp.FPN,
    'Linknet': smp.Linknet,
    'Unet': smp.Unet
}

if 'pretrained_model' in dict(conf):
    building_seg = get_model(conf.pretrained_model, 'torch', pretrained=conf.pretrained)
    preprocess_fn=None
else:
    building_seg = segmentation_types[conf.segmentation_arch](conf.encoder, classes=1, activation='sigmoid')
    preprocess_fn = get_preprocessing_fn(conf.encoder)

if conf.train_post:
    damage = DownscaleLayer(inplanes=building_seg.final_filters, blocks=conf.blocks, strides=conf.strides,
                           block=block_types[conf.blocktype],
                           nclasses=conf.n_dmg_classes,
                           growth_rate=conf.growth_rate)
else:
    damage = None

model = XVNet(building_seg, damage).to('cuda')

In [ ]:
from xv.nn import dataset
from xv import util
import random

dmg_downscale=1
for s in conf.strides:
    dmg_downscale *= s
    
dmg_downscale_ratio = dmg_downscale



instances = dataset.XViewSegmentationDataset.get_instances(train_dir)

random.seed(hash("😂"))
random.shuffle(instances)

dev_ix = int(len(instances)*.20)
dev_instances = instances[:dev_ix]
train_instances = instances[dev_ix:]
len(train_instances), len(dev_instances)

train_dataset = dataset.XViewSegmentationDataset(
    instances=train_instances,
    resolution=(conf.image_size, conf.image_size),
    dmg_downscale_ratio = dmg_downscale_ratio,
    augment=augment,
    damage_scale_mode=conf.damage_scale_mode,
    preprocess_fn=preprocess_fn,
)

dev_dataset = dataset.XViewSegmentationDataset(
    instances=dev_instances,
    resolution=(conf.image_size, conf.image_size),
    dmg_downscale_ratio = dmg_downscale_ratio,
    augment=None,
    damage_scale_mode=conf.damage_scale_mode,
    preprocess_fn=preprocess_fn
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=conf.batch_size,
    shuffle=True,
    num_workers=10,
    pin_memory=True,
)

dev_loader = torch.utils.data.DataLoader(
    dev_dataset,
    batch_size=conf.batch_size,
    shuffle=True,
    num_workers=10,
    pin_memory=True,
)

In [ ]:
from xv.nn.losses import loss_dict, WeightedLoss
loss = WeightedLoss({loss_dict[l]():w for l, w in conf.loss_weights.items()})

In [ ]:
import apex

optims = {
    'adam': torch.optim.Adam
}
optim = optims[conf.optim](model.parameters(), lr=conf.lr)

In [ ]:
from apex import amp
model, optim = amp.initialize(model, optim, opt_level=conf.amp_opt_level)

In [ ]:
wandb.watch(model);

In [ ]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, factor=conf.scheduler_factor, patience=conf.scheduler_patience)

In [ ]:
pre_weight = conf.pre_weight if conf.train_pre else None
post_weight = conf.post_weight if conf.train_post else None

In [ ]:
best_loss = 1e5
epoch = 0

In [ ]:
from xv import run
for epoch in range(epoch, conf.epochs):
    metrics = {'epoch':epoch}
    metrics.update((
        {f'train_{k}':v for k,v in
         run.train(model, optim, train_loader, loss, pre_weight=pre_weight, post_weight=post_weight).items()}
    ))
    metrics.update(
        run.evaluate(model, dev_loader, loss, pre_weight=pre_weight, post_weight=post_weight
    ))
    wandb.log(metrics)
    scheduler.step(metrics['loss'])
    if metrics['loss'] < best_loss:
        torch.save(model.state_dict(), os.path.join(wandb.run.dir, "state_dict.pth"))
        best_loss = metrics['loss']

In [ ]:
ix = 1000
i = train_dataset[ix]
images, masks = i['images'], i['masks']
image = images['post']
image = np.array(train_dataset.inverse_transform_image(image))

util.vis_im_mask(image, masks['damage'], size=(512*2,512*2), opacity=.3);

In [ ]:
from collections import Counter
counts = Counter(len(i['pre']['features']) for i in train_dataset.instances)